### Dependencies

In [3]:
# 📦 Install PyMuPDF if not already installed
!pip install pymupdf

import fitz  # PyMuPDF
import os
from urllib.parse import urlparse
import requests

### Download pdf from uri

In [ ]:
def download_pdf_from_uri(uri, download_folder="../sample_data"):
    parsed_uri = urlparse(uri)
    os.makedirs(download_folder, exist_ok=True)

    if parsed_uri.scheme in ['http', 'https']:
        filename = os.path.basename(parsed_uri.path)
        pdf_path = os.path.join(download_folder, filename)

        response = requests.get(uri)
        if response.status_code == 200:
            with open(pdf_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded PDF to {pdf_path}")
        else:
            raise Exception(f"Failed to download PDF. HTTP status: {response.status_code}")
    else:
        if not os.path.isfile(uri):
            raise FileNotFoundError(f"Local file not found: {uri}")
        pdf_path = uri

    return pdf_path

In [ ]:
pdf_uri = ""
pdf_file_path = download_pdf_from_uri(pdf_uri)

### Extract images from pdf

In [1]:
def extract_images_from_pdf(pdf_path, output_folder=None):
    if output_folder is None:
        pdf_name = os.path.splitext(os.path.basename(pdf_path))[0]
        output_folder = os.path.join(os.path.dirname(pdf_path), f"{pdf_name}_extracted_images")
    doc = fitz.open(pdf_path)
    os.makedirs(output_folder, exist_ok=True)

    image_count = 0
    for page_num in range(len(doc)):
        page = doc[page_num]
        images = page.get_images(full=True)

        for img_index, img in enumerate(images):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]

            image_filename = f"image_page{page_num+1}_{img_index+1}.{image_ext}"
            image_path = os.path.join(output_folder, image_filename)

            with open(image_path, "wb") as f:
                f.write(image_bytes)
                image_count += 1

    print(f"✅ Extracted {image_count} images to '{output_folder}'")



In [4]:
pdf_file_path = "../raw_data/Certificado_UC3M.pdf"
extract_images_from_pdf(pdf_file_path)

MuPDF error: format error: cmsOpenProfileFromMem failed

✅ Extracted 18 images to '../raw_data/Certificado_UC3M_extracted_images'
